In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
mako = sns.color_palette("mako", as_cmap=True)

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()

from tqdm.notebook import tqdm
import os
from multiprocess import Pool
try:
    num_cpus = len(os.sched_getaffinity(0))
except:
    num_cpus = os.cpu_count()
print("Using {} CPUs".format(num_cpus))

import sys
sys.path.insert(0, 'toric-decoder')
from toric import *
from pymatching import Matching

Using 8 CPUs


In [39]:
# Simulation parameters

L = 10 # Lattice size
p_error = 0.25 # Error probability per spin
η = 0.1 # Smoothing paramter for Jacobi method
c = 10 # "Field velocity" - number of field updates per cycle
T = L # Epochs

matching = Matching(pcm(L))

In [46]:
mystate = init_state(L, p_error)
q_history, error_history = decoder_2D(mystate, T, c, η, history = True)
print("Fail after 2D: {}".format(mystate.N > 0 or logical_error(mystate.error)))
correction = mwpm(matching, q_history[-1,:,:])
np.append(q_history, [np.zeros_like(mystate.q)], axis = 0)
np.append(error_history, [mystate.error ^ correction], axis = 0)
print("Fail after MWPM: {}".format(logical_error(error_history[-1,:,:,:])))

Fail after 2D: True
Fail after MWPM: True


In [47]:
plot_evolution(q_history, error_history, dual = True)

## Parameters
* System size $L\in\left\{20,40,\dotsc,100\right\}$
* Field velocity $c\in\left\{2,4,\dotsc,32\right\}$
* Error rate per $\tau$ time steps: $p=0.05$, $\tau=1$
* Time steps $T=10L$
* Smoothing factor $\eta=0.1$
* Shots: $10^4$
## Data
* Failure rate
* Density of anyons $N/L^2$

In [5]:
fail_rate = np.ones((60, 60)) # L/2 - 1, c - 1
shots = 1 # error per pixel is max. +/- 0.05

In [6]:
for c in tqdm(range(1, 61)):
    for L in range(2, 122, 2):
        T = 10 * L
        fails = 0
        for _ in range(shots):
            mystate = init_state(L, p_error)
            decoder_2D(mystate, T, c, η, history = False)
            fails += (mystate.N > 0 or logical_error(mystate.error))
        fail_rate[L // 2 - 1, c - 1] = fails / shots

np.save("fail_rate.npy", fail_rate)

  0%|          | 0/60 [00:00<?, ?it/s]

In [ ]:
from matplotlib.colors import LogNorm
fig, ax = plt.subplots()
mat = ax.matshow(fail_rate.T, origin = 'lower', cmap = mako, norm = LogNorm())
fig.colorbar(mat)
ax.set_xlabel('System size $L/2-1$')
ax.set_ylabel('Field velocity $c-1$')
ax.xaxis.tick_bottom()
ax.set_title('Failure rate')
plt.savefig('fail_rate.png')
plt.show()